In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Rondônia - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_RO.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rondônia - Produção de Cimento (t),Rondônia - IDH,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - Desemprego,Rondônia - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,4.987000,0.688843,2.068350e+07,1.587672e+06,11.468162,1.828350e+07,8.226062,13.867
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,4.987000,0.689031,2.071062e+07,1.589518e+06,11.472453,1.829323e+07,8.219943,10.777
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,4.987000,0.689218,2.073774e+07,1.591364e+06,11.476744,1.830296e+07,8.213823,13.888
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,4.987000,0.689405,2.076486e+07,1.593210e+06,11.481034,1.831268e+07,8.207703,11.859
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,4.987000,0.689592,2.079198e+07,1.595056e+06,11.485325,1.832241e+07,8.201584,14.431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.931530,NaN,NaN,NaN,NaN,NaN,NaN,49.066
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.812259,NaN,NaN,NaN,NaN,NaN,NaN,49.850
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.687985,NaN,NaN,NaN,NaN,NaN,NaN,43.733
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.551024,NaN,NaN,NaN,NaN,NaN,NaN,40.078


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
# mean = np.mean(input_data, axis=0)
# stddev =  np.std(input_data, axis=0)
# input_data = ((input_data - mean) /stddev)
scaler=MinMaxScaler()
scaled_data = scaler.fit_transform(input_data)
input_data = pd.DataFrame(scaled_data, columns=input_data.columns)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rondônia - Produção de Cimento (t),Rondônia - IDH,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - Desemprego
0,1.000000,1.000000,0.746035,1.000000,0.000000,0.000000,1.000000,1.387779e-17,0.000000,0.000000,0.712990,0.000000,0.000000,1.000000
1,0.921240,0.965719,0.580259,0.936603,0.007992,0.011007,0.968938,1.387779e-17,0.007183,0.006044,0.722552,0.011690,0.009174,0.996356
2,0.873107,0.927412,0.926926,0.906543,0.015984,0.022015,0.948438,1.387779e-17,0.014365,0.012088,0.732115,0.023380,0.018349,0.992713
3,0.851659,0.885588,0.549924,0.858675,0.023976,0.033022,0.928794,1.387779e-17,0.021548,0.018132,0.741677,0.035070,0.027523,0.989069
4,0.823277,0.838034,0.807277,0.824839,0.031968,0.044030,0.970421,1.387779e-17,0.028730,0.024176,0.751239,0.046760,0.036698,0.985426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.000000,0.030808,0.063780,0.381401,0.987600,0.828698,0.005731,2.962589e-01,0.946574,0.964247,0.122595,0.951625,0.940149,0.309246
188,0.029504,0.027720,0.048101,0.466609,0.990700,0.823090,0.037761,2.988317e-01,0.942025,0.961372,0.129858,0.946873,0.936905,0.301906
189,0.043125,0.023074,0.016800,0.530936,0.993800,0.817482,0.087044,3.062301e-01,0.937476,0.958497,0.137120,0.942120,0.933661,0.294567
190,0.059756,0.017547,0.035186,0.611274,0.996900,0.811875,0.118264,3.059535e-01,0.932927,0.955622,0.144383,0.937368,0.930417,0.287228


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      13.420
1      11.302
2      16.993
3      16.214
4      15.406
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Rondônia - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rondônia - Produção de Cimento (t),Rondônia - IDH,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - Desemprego
0,1.000000,1.000000,0.746035,1.000000,0.000000,0.000000,1.000000,1.387779e-17,0.000000,0.000000,0.712990,0.000000,0.000000,1.000000
1,0.921240,0.965719,0.580259,0.936603,0.007992,0.011007,0.968938,1.387779e-17,0.007183,0.006044,0.722552,0.011690,0.009174,0.996356
2,0.873107,0.927412,0.926926,0.906543,0.015984,0.022015,0.948438,1.387779e-17,0.014365,0.012088,0.732115,0.023380,0.018349,0.992713
3,0.851659,0.885588,0.549924,0.858675,0.023976,0.033022,0.928794,1.387779e-17,0.021548,0.018132,0.741677,0.035070,0.027523,0.989069
4,0.823277,0.838034,0.807277,0.824839,0.031968,0.044030,0.970421,1.387779e-17,0.028730,0.024176,0.751239,0.046760,0.036698,0.985426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.379453,0.024030,0.338208,0.039971,0.890665,0.956993,0.182507,3.510513e-01,0.999974,0.980511,0.001994,0.941484,0.976362,0.635312
158,0.332832,0.021531,0.371581,0.027548,0.893949,0.953662,0.168812,3.445820e-01,0.999948,0.982282,0.003987,0.946804,0.978511,0.617372
159,0.313954,0.019385,0.365349,0.037038,0.897232,0.950331,0.152959,3.382570e-01,0.999922,0.984054,0.005981,0.952124,0.980660,0.599432
160,0.294856,0.017671,0.404993,0.070841,0.900516,0.947000,0.137613,3.329312e-01,0.999895,0.985826,0.007974,0.957443,0.982809,0.581492


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      13.420
1      11.302
2      16.993
3      16.214
4      15.406
        ...  
157    29.662
158    31.408
159    29.484
160    33.601
161    36.905
Name: Rondônia - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 14)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rondônia - Produção de Cimento (t),Rondônia - IDH,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - Desemprego
126,0.612440,0.052718,0.094523,0.201912,0.780557,0.994213,0.372054,0.749278,0.702585,0.829330,0.294337,0.780042,0.859057,0.330889
127,0.626227,0.056653,0.200555,0.209545,0.784414,0.995177,0.375438,0.729191,0.707741,0.836097,0.284102,0.785829,0.864351,0.349248
128,0.645442,0.060771,0.214993,0.238038,0.788271,0.996142,0.372254,0.705470,0.712896,0.842863,0.273868,0.791616,0.869646,0.367608
129,0.673151,0.065105,0.053026,0.249843,0.792128,0.997106,0.369458,0.688587,0.718052,0.849630,0.263634,0.797403,0.874940,0.385968
130,0.699949,0.069099,0.016760,0.266770,0.795984,0.998071,0.371547,0.670094,0.723207,0.856397,0.253399,0.803190,0.880234,0.404328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.000000,0.030808,0.063780,0.381401,0.987600,0.828698,0.005731,0.296259,0.946574,0.964247,0.122595,0.951625,0.940149,0.309246
188,0.029504,0.027720,0.048101,0.466609,0.990700,0.823090,0.037761,0.298832,0.942025,0.961372,0.129858,0.946873,0.936905,0.301906
189,0.043125,0.023074,0.016800,0.530936,0.993800,0.817482,0.087044,0.306230,0.937476,0.958497,0.137120,0.942120,0.933661,0.294567
190,0.059756,0.017547,0.035186,0.611274,0.996900,0.811875,0.118264,0.305953,0.932927,0.955622,0.144383,0.937368,0.930417,0.287228


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 14)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
#     optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(36, activation='tanh', 
                                 return_sequences=False), 
                                 input_shape=(train_input.shape[1], 
                                              train_input.shape[2])),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(180, activation='tanh')),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(5)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, 
                              reshaped_target, 
                              reshaped_test, 
                              reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[686221245, 542140766, 3080224446, 1678001866, 1606619940]


Step: 0 ___________________________________________
val_loss: 38.82969665527344
winner_seed: 686221245


Step: 1 ___________________________________________
val_loss: 39.214996337890625


Step: 2 ___________________________________________
val_loss: 34.58176040649414
winner_seed: 3080224446


Step: 3 ___________________________________________
val_loss: 31.845428466796875
winner_seed: 1678001866


Step: 4 ___________________________________________
val_loss: 37.71525573730469


final_seed: 1678001866


In [35]:
trained_model, history = bidirectional_lstm_model(reshaped_train, 
                                                  reshaped_target, 
                                                  want_verbose=1, 
                                                  seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 109ms/step - loss: 3264.0359 - val_loss: 2975.3420
Epoch 2/10000
4/4 [==============================] - 0s 7ms/step - loss: 2882.8564 - val_loss: 2640.1479
Epoch 3/10000
4/4 [==============================] - 0s 7ms/step - loss: 2562.5212 - val_loss: 2361.4424
Epoch 4/10000
4/4 [==============================] - 0s 7ms/step - loss: 2293.2324 - val_loss: 2127.2080
Epoch 5/10000
4/4 [==============================] - 0s 7ms/step - loss: 2066.9607 - val_loss: 1939.1970
Epoch 6/10000
4/4 [==============================] - 0s 7ms/step - loss: 1890.2472 - val_loss: 1789.6470
Epoch 7/10000
4/4 [==============================] - 0s 7ms/step - loss: 1748.7045 - val_loss: 1666.7308
Epoch 8/10000
4/4 [==============================] - 0s 7ms/step - loss: 1629.3619 - val_loss: 1559.8732
Epoch 9/10000
4/4 [==============================] - 0s 7ms/step - loss: 1525.7108 - val_loss: 1462.6073
Epoch 10/10000
4/4 [==============================] -

4/4 [==============================] - 0s 7ms/step - loss: 494.6437 - val_loss: 533.2307
Epoch 80/10000
4/4 [==============================] - 0s 7ms/step - loss: 494.6415 - val_loss: 533.2596
Epoch 81/10000
4/4 [==============================] - 0s 7ms/step - loss: 494.5088 - val_loss: 533.2517
Epoch 82/10000
4/4 [==============================] - 0s 7ms/step - loss: 494.6412 - val_loss: 533.2051
Epoch 83/10000
4/4 [==============================] - 0s 7ms/step - loss: 494.5280 - val_loss: 533.2072
Epoch 84/10000
4/4 [==============================] - 0s 7ms/step - loss: 494.7027 - val_loss: 533.2756
Epoch 85/10000
4/4 [==============================] - 0s 7ms/step - loss: 494.5564 - val_loss: 533.2913
Epoch 86/10000
4/4 [==============================] - 0s 7ms/step - loss: 494.4295 - val_loss: 533.2634
Epoch 87/10000
4/4 [==============================] - 0s 7ms/step - loss: 494.6561 - val_loss: 533.1818
Epoch 88/10000
4/4 [==============================] - 0s 7ms/step - loss: 494.4

4/4 [==============================] - 0s 7ms/step - loss: 119.5359 - val_loss: 172.8050
Epoch 158/10000
4/4 [==============================] - 0s 7ms/step - loss: 119.8194 - val_loss: 168.4714
Epoch 159/10000
4/4 [==============================] - 0s 7ms/step - loss: 117.8812 - val_loss: 177.9109
Epoch 160/10000
4/4 [==============================] - 0s 7ms/step - loss: 117.0026 - val_loss: 164.9175
Epoch 161/10000
4/4 [==============================] - 0s 7ms/step - loss: 115.6700 - val_loss: 163.1458
Epoch 162/10000
4/4 [==============================] - 0s 7ms/step - loss: 113.7348 - val_loss: 166.5207
Epoch 163/10000
4/4 [==============================] - 0s 7ms/step - loss: 111.4040 - val_loss: 160.8682
Epoch 164/10000
4/4 [==============================] - 0s 7ms/step - loss: 110.5725 - val_loss: 163.9338
Epoch 165/10000
4/4 [==============================] - 0s 7ms/step - loss: 108.5842 - val_loss: 165.6741
Epoch 166/10000
4/4 [==============================] - 0s 7ms/step - lo

Epoch 236/10000
4/4 [==============================] - 0s 7ms/step - loss: 71.9548 - val_loss: 123.2969
Epoch 237/10000
4/4 [==============================] - 0s 7ms/step - loss: 74.4841 - val_loss: 116.8904
Epoch 238/10000
4/4 [==============================] - 0s 7ms/step - loss: 71.5682 - val_loss: 123.4685
Epoch 239/10000
4/4 [==============================] - 0s 7ms/step - loss: 70.7502 - val_loss: 115.0845
Epoch 240/10000
4/4 [==============================] - 0s 7ms/step - loss: 75.0928 - val_loss: 108.7138
Epoch 241/10000
4/4 [==============================] - 0s 7ms/step - loss: 78.4998 - val_loss: 114.8590
Epoch 242/10000
4/4 [==============================] - 0s 7ms/step - loss: 70.1643 - val_loss: 119.1960
Epoch 243/10000
4/4 [==============================] - 0s 7ms/step - loss: 69.2494 - val_loss: 112.7655
Epoch 244/10000
4/4 [==============================] - 0s 7ms/step - loss: 68.5755 - val_loss: 120.6379
Epoch 245/10000
4/4 [==============================] - 0s 7ms/st

Epoch 315/10000
4/4 [==============================] - 0s 7ms/step - loss: 61.1031 - val_loss: 104.1490
Epoch 316/10000
4/4 [==============================] - 0s 7ms/step - loss: 59.1699 - val_loss: 98.8536
Epoch 317/10000
4/4 [==============================] - 0s 7ms/step - loss: 58.6970 - val_loss: 97.2287
Epoch 318/10000
4/4 [==============================] - 0s 7ms/step - loss: 58.8384 - val_loss: 102.8381
Epoch 319/10000
4/4 [==============================] - 0s 7ms/step - loss: 54.8695 - val_loss: 104.9477
Epoch 320/10000
4/4 [==============================] - 0s 7ms/step - loss: 59.2165 - val_loss: 100.7632
Epoch 321/10000
4/4 [==============================] - 0s 7ms/step - loss: 56.4272 - val_loss: 104.0594
Epoch 322/10000
4/4 [==============================] - 0s 7ms/step - loss: 56.8165 - val_loss: 98.9697
Epoch 323/10000
4/4 [==============================] - 0s 7ms/step - loss: 55.4748 - val_loss: 99.3269
Epoch 324/10000
4/4 [==============================] - 0s 7ms/step -

...

4/4 [==============================] - 0s 7ms/step - loss: 2.1747 - val_loss: 36.3352
Epoch 2606/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.0445 - val_loss: 39.9579
Epoch 2607/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.4026 - val_loss: 44.4946
Epoch 2608/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.3369 - val_loss: 40.6998
Epoch 2609/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.2512 - val_loss: 36.5319
Epoch 2610/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.1460 - val_loss: 36.6292
Epoch 2611/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.9978 - val_loss: 37.3440
Epoch 2612/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.9191 - val_loss: 37.6528
Epoch 2613/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.7098 - val_loss: 36.6001
Epoch 2614/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.6292 - val_l

4/4 [==============================] - 0s 7ms/step - loss: 62.5592 - val_loss: 73.6803
Epoch 2685/10000
4/4 [==============================] - 0s 7ms/step - loss: 61.3092 - val_loss: 89.1029
Epoch 2686/10000
4/4 [==============================] - 0s 7ms/step - loss: 62.9447 - val_loss: 69.5196
Epoch 2687/10000
4/4 [==============================] - 0s 7ms/step - loss: 53.1463 - val_loss: 63.0940
Epoch 2688/10000
4/4 [==============================] - 0s 7ms/step - loss: 41.6513 - val_loss: 53.6607
Epoch 2689/10000
4/4 [==============================] - 0s 7ms/step - loss: 34.6245 - val_loss: 46.2717
Epoch 2690/10000
4/4 [==============================] - 0s 7ms/step - loss: 34.6112 - val_loss: 43.5615
Epoch 2691/10000
4/4 [==============================] - 0s 7ms/step - loss: 33.4948 - val_loss: 47.0122
Epoch 2692/10000
4/4 [==============================] - 0s 7ms/step - loss: 32.7432 - val_loss: 50.4377
Epoch 2693/10000
4/4 [==============================] - 0s 7ms/step - loss: 32.71

4/4 [==============================] - 0s 7ms/step - loss: 8.0871 - val_loss: 43.3312
Epoch 2764/10000
4/4 [==============================] - 0s 7ms/step - loss: 12.2919 - val_loss: 33.0482
Epoch 2765/10000
4/4 [==============================] - 0s 7ms/step - loss: 14.3775 - val_loss: 36.9885
Epoch 2766/10000
4/4 [==============================] - 0s 7ms/step - loss: 9.7256 - val_loss: 61.5418
Epoch 2767/10000
4/4 [==============================] - 0s 7ms/step - loss: 20.3632 - val_loss: 54.7680
Epoch 2768/10000
4/4 [==============================] - 0s 7ms/step - loss: 28.4606 - val_loss: 54.7272
Epoch 2769/10000
4/4 [==============================] - 0s 7ms/step - loss: 26.6765 - val_loss: 61.7032
Epoch 2770/10000
4/4 [==============================] - 0s 7ms/step - loss: 40.5044 - val_loss: 68.9770
Epoch 2771/10000
4/4 [==============================] - 0s 7ms/step - loss: 46.7609 - val_loss: 64.7182
Epoch 2772/10000
4/4 [==============================] - 0s 7ms/step - loss: 45.2137

4/4 [==============================] - 0s 7ms/step - loss: 76.2128 - val_loss: 52.1733
Epoch 2843/10000
4/4 [==============================] - 0s 7ms/step - loss: 32.1356 - val_loss: 88.4435
Epoch 2844/10000
4/4 [==============================] - 0s 7ms/step - loss: 90.2899 - val_loss: 179.8730
Epoch 2845/10000
4/4 [==============================] - 0s 7ms/step - loss: 100.0372 - val_loss: 144.6978
Epoch 2846/10000
4/4 [==============================] - 0s 7ms/step - loss: 89.9653 - val_loss: 128.1854
Epoch 2847/10000
4/4 [==============================] - 0s 7ms/step - loss: 92.1011 - val_loss: 124.6925
Epoch 2848/10000
4/4 [==============================] - 0s 7ms/step - loss: 84.6816 - val_loss: 123.1926
Epoch 2849/10000
4/4 [==============================] - 0s 7ms/step - loss: 72.1376 - val_loss: 126.5055
Epoch 2850/10000
4/4 [==============================] - 0s 7ms/step - loss: 66.8096 - val_loss: 129.6975
Epoch 2851/10000
4/4 [==============================] - 0s 7ms/step - los

4/4 [==============================] - 0s 7ms/step - loss: 34.8277 - val_loss: 77.6452
Epoch 2921/10000
4/4 [==============================] - 0s 7ms/step - loss: 35.9728 - val_loss: 79.6397
Epoch 2922/10000
4/4 [==============================] - 0s 7ms/step - loss: 30.5456 - val_loss: 76.2490
Epoch 2923/10000
4/4 [==============================] - 0s 7ms/step - loss: 30.2342 - val_loss: 73.1490
Epoch 2924/10000
4/4 [==============================] - 0s 7ms/step - loss: 29.5495 - val_loss: 72.6753
Epoch 2925/10000
4/4 [==============================] - 0s 7ms/step - loss: 29.5634 - val_loss: 72.0605
Epoch 2926/10000
4/4 [==============================] - 0s 7ms/step - loss: 26.9033 - val_loss: 72.9909
Epoch 2927/10000
4/4 [==============================] - 0s 7ms/step - loss: 28.4934 - val_loss: 73.6685
Epoch 2928/10000
4/4 [==============================] - 0s 7ms/step - loss: 28.0158 - val_loss: 74.2182
Epoch 2929/10000
4/4 [==============================] - 0s 7ms/step - loss: 27.72

In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 13ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,32.129601,31.649559,32.631428,32.816574,33.165405,32.830078,33.043995,33.857899,34.278751,34.341251,34.607891,35.771526,33.692448,34.302757,35.934715,35.446342,32.628063,29.960659,31.642397,35.402393,36.213898,39.01907,37.975769,40.988541,40.949791,33.259342,36.930782,40.901016,42.032536,41.03027
Target,36.094,42.556,41.193,37.053,33.527,25.616,25.221,23.321,23.927,24.561,20.135,34.837,34.338,39.293,41.61,43.544,35.658,29.693,32.559,24.91,28.713,32.66,31.635,35.142,36.419,40.3,35.493,38.489,35.591,27.53
Error,3.964401,10.906441,8.561573,4.236427,0.361595,7.214079,7.822994,10.5369,10.351751,9.780251,14.472891,0.934525,0.645554,4.990242,5.675285,8.097656,3.029938,0.267658,0.916601,10.492393,7.500898,6.35907,6.340769,5.846542,4.530792,7.040657,1.437782,2.412018,6.441536,13.500269


In [38]:
display(mae)
display(mape)

6.1556487

0.20756929

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[401.12393]] - Target[368.041]| =  Error: [[33.082947]]; MAPE:[[0.0898893]]
1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[423.2071]] - Target[409.75500000000005]| =  Error: [[13.452087]]; MAPE:[[0.03282959]]
1/1 [==============================] - 0s 13ms/step
Ano-5: |Prediction[[235.10374]] - Target[213.822]| =  Error: [[21.281738]]; MAPE:[[0.09953016]]


[array([[33.082947]], dtype=float32),
 array([[13.452087]], dtype=float32),
 array([[21.281738]], dtype=float32)]

22.60559

0.074083015